In [2]:
import pandas as pd
import numpy as np
import json
import gzip
import ast
from textblob import TextBlob

In [ ]:
#json_data = ast.literal_eval(line)  esta es la linia clave

archivo_items = "users_items.json.gz"
data_items = []

with gzip.open(archivo_items, 'rt', encoding='utf-8') as file:
    for line in file:
        try:
            json_data = ast.literal_eval(line)
            data_items.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

df_user_items = pd.DataFrame(data_items)

#toma 3m 10s

In [3]:
# Leer el archivo Parquet y cargarlo en un DataFrame
df_user_items = pd.read_parquet('australian_users_items.parquet')


#toma 7.1s

In [3]:
df_user_items.tail()

,user_id,items_count,steam_id,user_url,items
88305,76561198323066619,22,76561198323066624,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700688,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759264,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548336,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."
88309,edward_tremethick,0,76561198331598576,http://steamcommunity.com/id/edward_tremethick,[]


In [4]:
#df_user_items['items'][0]
# Extraer la columna 'items' como una lista de listas
items_list = df_user_items['items'][0].tolist()
c=0
# Imprimir cada sublista en una nueva línea
for sublist in items_list:
    print(sublist)
    c=c+1
print(c)
# Sumar todos los números de la columna 'items_count'
total_items_count = df_user_items['items_count'].sum()

# Imprimir el resultado
print("La suma total de items_count es:", total_items_count)

{'item_id': '10', 'item_name': 'Counter-Strike', 'playtime_2weeks': 0, 'playtime_forever': 6}
{'item_id': '20', 'item_name': 'Team Fortress Classic', 'playtime_2weeks': 0, 'playtime_forever': 0}
{'item_id': '30', 'item_name': 'Day of Defeat', 'playtime_2weeks': 0, 'playtime_forever': 7}
{'item_id': '40', 'item_name': 'Deathmatch Classic', 'playtime_2weeks': 0, 'playtime_forever': 0}
{'item_id': '50', 'item_name': 'Half-Life: Opposing Force', 'playtime_2weeks': 0, 'playtime_forever': 0}
{'item_id': '60', 'item_name': 'Ricochet', 'playtime_2weeks': 0, 'playtime_forever': 0}
{'item_id': '70', 'item_name': 'Half-Life', 'playtime_2weeks': 0, 'playtime_forever': 0}
{'item_id': '130', 'item_name': 'Half-Life: Blue Shift', 'playtime_2weeks': 0, 'playtime_forever': 0}
{'item_id': '300', 'item_name': 'Day of Defeat: Source', 'playtime_2weeks': 0, 'playtime_forever': 4733}
{'item_id': '240', 'item_name': 'Counter-Strike: Source', 'playtime_2weeks': 0, 'playtime_forever': 1853}
{'item_id': '3830',

In [4]:
print(df_user_items.isnull().sum())

user_id        0
items_count    0
steam_id       0
user_url       0
items          0
dtype: int64


In [6]:
df_user_items.shape

(88310, 5)

In [5]:
duplicados = df_user_items.duplicated(subset=['user_id', 'items_count','steam_id','user_url'])
#'tags','genres' exclui las lista por que generava error y no me parecia demaciado relevante 
 
print(f"Número de filas duplicadas: {(duplicados).sum()}")

Número de filas duplicadas: 684


In [7]:
df_user_items = df_user_items[~duplicados]

In [8]:
# Expandir las listas de reseñas en filas individuales
items_expanded = df_user_items.explode('items').reset_index(drop=True)

In [9]:
items_expanded.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982480,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike..."
1,76561197970982479,277,76561197970982480,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ..."
2,76561197970982479,277,76561197970982480,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
3,76561197970982479,277,76561197970982480,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
4,76561197970982479,277,76561197970982480,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."


In [10]:
items_expanded.shape

(5110796, 5)

In [11]:
items_expanded['items'][0]

{'item_id': '10',
 'item_name': 'Counter-Strike',
 'playtime_2weeks': 0,
 'playtime_forever': 6}

In [12]:
items_expanded = items_expanded.drop(columns=['items_count', 'steam_id','user_url'])

In [13]:
items_expanded.head()

,user_id,items
0,76561197970982479,"{'item_id': '10', 'item_name': 'Counter-Strike..."
1,76561197970982479,"{'item_id': '20', 'item_name': 'Team Fortress ..."
2,76561197970982479,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
3,76561197970982479,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
4,76561197970982479,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."


In [ ]:
# Guardar el DataFrame como un archivo Parquet comprimido
items_expanded.to_parquet('item.parquet', compression='snappy')

In [ ]:
import os
from fastapi import FastAPI, Request, Query, Path, HTTPException
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

app = FastAPI(title='Proyecto Integrador I Hecho por Michael Martinez')
templates = Jinja2Templates(directory="templates")

# Cargar y procesar datos al inicio de la aplicación
df_recom = pd.read_parquet(r'https://github.com/bkmay1417/Machine-Learning-Operations-MLOps-/blob/4e23689d2e1a74fddcc9251cee8614c16618cbcf/Dataset/recomendacion.parquet?raw=True')

# Vectorizar los géneros
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_recom['genres_str'])

# Calcular la similitud del coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

@app.get("/", tags=['Página Principal'])
async def read_root(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})

@app.get("/consulta1")
async def developer(developer: str = Query(default='Monster Games')):
    df_games = pd.read_parquet(r'https://github.com/bkmay1417/Machine-Learning-Operations-MLOps-/blob/82702a42172b2b0f23c1e24c6f9fdb294c52d78e/Dataset/developer.parquet?raw=True')
    df_filtrado = df_games[df_games['developer'] == developer]

    conteo_por_año = df_filtrado.groupby('release_date').size().reset_index(name='Cantidad de Items')
    conteo_free_to_play_por_año = df_filtrado[df_filtrado['price'] == 0.00].groupby('release_date').size().reset_index(name='free_to_play_games')

    df_resultado = pd.merge(conteo_por_año, conteo_free_to_play_por_año, on='release_date', how='left')
    df_resultado['Contenido Free'] = ((df_resultado['free_to_play_games'] / df_resultado['Cantidad de Items']) * 100).map('{:.2f}%'.format)
    df_resultado = df_resultado.drop(columns=['free_to_play_games'])
    df_resultado['Contenido Free'] = df_resultado['Contenido Free'].replace('nan%', '0%')

    resultado_dict = df_resultado.to_dict(orient='records')
    return resultado_dict

@app.get("/consulta2")
async def userdata():
    return {}

@app.get("/consulta3")
async def UserForGenre():
    return {}

@app.get("/consulta4")
async def best_developer_year(year: int = Query(default=2005)):
    merged_df = pd.read_parquet(r'https://github.com/bkmay1417/Machine-Learning-Operations-MLOps-/blob/8f87ccc010ef4ab3025d5e95d5f0cc1ee11fd276/Dataset/best_developer_year.parquet?raw=True')
    merged_df = merged_df[merged_df['release_date'] == year]
    developer_counts = merged_df['developer'].value_counts()
    top_developers = developer_counts.head(3).index

    result = [{f"Puesto {i + 1}": developer} for i, developer in enumerate(top_developers)]
    return result

@app.get("/consulta5")
async def developer_reviews_analysis(desarrolladora: str = Query(default='Valve')):
    reviews = pd.read_parquet(r'https://github.com/bkmay1417/Machine-Learning-Operations-MLOps-/blob/8f87ccc010ef4ab3025d5e95d5f0cc1ee11fd276/Dataset/reviews_analysis.parquet?raw=True')
    reviews = reviews[reviews['developer'] == desarrolladora]
    counts = reviews['sentiment_analysis'].value_counts()

    resultado = {
        desarrolladora: [
            f"Negative = {counts.get(0, 0)}",
            f"Positive = {counts.get(2, 0)}"
        ]
    }
    return resultado

@app.get("/Sistema de recomendacion")
async def recomendacion_juego(item_id: float = Query(default=10.0)):
    if item_id not in df_recom['item_id'].values:
        raise HTTPException(status_code=404, detail="Item ID no encontrado")

    idx = df_recom[df_recom['item_id'] == item_id].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]

    game_indices = [i[0] for i in sim_scores]
    return df_recom['title'].iloc[game_indices].tolist()

if __name__ == "__main__":
    port = int(os.environ.get("PORT", 8000))
    uvicorn.run(app, host="0.0.0.0", port=port)
